In [ ]:
!pip install scikit-plot -q

In [212]:
import openpyxl #biblioteca para abrir arquivos em excel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV # biblioteca para treino da arvore de decisão e otimização de hiperparâmetros
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor #para classificação e regressão as variáveis
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # biblioteca para métricas do modelo
from sklearn.ensemble import RandomForestClassifier
#bibliotecas para visualização das árvore arvore de decisão
from sklearn.tree import export_text
from sklearn import tree
from IPython.display import Image  
from io import StringIO 
from sklearn.tree import export_graphviz
import pydot 
from sklearn.datasets import load_iris
import graphviz

In [213]:
sns.set_style()
pd.set_option('display.max_columns', 25)

In [214]:
#carregando os dados
bd1 = pd.read_excel("defaultCreditcardClients.xls")

In [ ]:
# Aqui retiramos a linha onde tinha apenas o nome da variável, pois esta estava sendo contada como valor sem o ser.
#Logo, bd1_copy é o mesmo bd1 só que sem os cabeçalhos descritivos
bd1_copy = bd1.iloc[1:]
bd1_copy

In [223]:
# Dividindo os dados em features eixo(X) e variável alvo eixo(y)
X = bd1_copy.drop(['Y'],  axis=1)
y = bd1_copy['Y'].astype(int)

In [230]:
# Dividindo os dados em features eixo(X) e variável alvo eixo(y)
X = bd1_copy.drop(columns=['Y'],  axis=1)
y = bd1_copy['Y'].astype(int)

In [224]:
# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)